In [2]:
import sys
import numpy as np
import math
import random
import json
import requests
import random
import numpy as np
import gym

sys.path.append('c://Users/omara/Desktop/VSCODE/DigitalSquad/gym-maze/')
import gym_maze
from gym_maze.envs.maze_manager import MazeManager
from riddle_solvers import *


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from collections import deque

In [3]:
#pytorch nn for q network

class QNetwork(nn.Module):
    def model(self):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(3, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 4)

    def forward(self, x):
        

In [4]:
def train_dqn(env, model, optimizer, num_episodes, gamma=0.99):
    for i in range(num_episodes):
        obs = env.reset()
        done = False
        while not done:
            action = model.act(obs)
            next_obs, reward, done, _, info = env.step(action)
            model.remember(obs, action, reward, next_obs, done)
            obs = next_obs
            model.learn(gamma, optimizer)

def run_dqn(env, model):
    obs = env.reset()
    done = False
    while not done:
        action = model.act(obs)
        obs, reward, done, _ = env.step(action)
        env.render()
    env.close()


In [9]:
# continue from where we left off

def main():

    # initialize maze manager and environment
    manager = MazeManager()
    manager.init_maze(9, maze_cells=10)
    env = manager.maze_map[9]

    # set hyperparameters
    num_episodes = 500
    batch_size = 32
    gamma = 0.99
    eps_start = 1.0
    eps_end = 0.01
    eps_decay = 0.0001
    target_update = 10

    # initialize DQN agent and target network
    policy_net = DQN(env.observation_space.shape[0], env.action_space.n).to(device)
    target_net = DQN(env.observation_space.shape[0], env.action_space.n).to(device)
    target_net.load_state_dict(policy_net.state_dict())
    target_net.eval()

    optimizer = optim.Adam(policy_net.parameters())
    memory = ReplayMemory(10000)

    steps_done = 0

    # function to select action using epsilon-greedy policy
    def select_action(state, eps):
        global steps_done
        sample = random.random()
        eps_threshold = eps_end + (eps_start - eps_end) * math.exp(-1.0 * steps_done * eps_decay)
        steps_done += 1
        if sample > eps_threshold:
            with torch.no_grad():
                return policy_net(state).max(1)[1].view(1, 1)
        else:
            return torch.tensor([[random.randrange(env.action_space.n)]], device=device, dtype=torch.long)

    # function to optimize model parameters using backpropagation
    def optimize_model():
        if len(memory) < batch_size:
            return
        transitions = memory.sample(batch_size)
        batch = Transition(*zip(*transitions))

        non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), device=device, dtype=torch.uint8)
        non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)

        state_action_values = policy_net(state_batch).gather(1, action_batch)

        next_state_values = torch.zeros(batch_size, device=device)
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
        expected_state_action_values = (next_state_values * gamma) + reward_batch

        loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

        optimizer.zero_grad()
        loss.backward()
        for param in policy_net.parameters():
            param.grad.data.clamp_(-1, 1)
        optimizer.step()

    # training loop
    for i_episode in range(num_episodes):

        # reset environment and initialize state
        state = env.reset()
        state = torch.tensor(state, device=device, dtype=torch.float32).unsqueeze(0)

        for t in count():

            # select and perform action
            action = select_action(state, eps_start)
            next_state, reward, done, _ = env.step(action.item())
            reward = torch.tensor([reward], device=device)

            if not done:
                next_state = torch.tensor(next_state, device=device, dtype=torch.float32).unsqueeze(0)
            else:
                next_state = None

            # store transition in memory
            memory.push(state, action, next_state, reward)

            # move to next state
            state = next_state


In [10]:
main()

Exception: Enter a Numpy array!